# FIFA 23 PREDICTIVE AI 

In [323]:
# AUTHOR: Kobina Kyereboah-Coleman
# DATE: June 23rd 2024
# TOD: Assignment 2
# TODO: Player Rating Prediction AI

## PYTHON LIBRARIES & MODULES

In [324]:
import pandas as pd
import numpy as np
import os
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

## DATA PREP

In [325]:
# Load the dataset 
fifa_23 = pd.read_csv('/Users/kobby/Downloads/Male Players Legacy.csv')

/var/folders/23/2w1hzc8j4qj80m0y2t4jqwvh0000gn/T/ipykernel_1019/4126988420.py:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa_23 = pd.read_csv('/Users/kobby/Downloads/Male Players Legacy.csv')


In [326]:
# View the first 5 rows of the dataframe
fifa_23.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [327]:
fifa_23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 110 entries, player_id to player_face_url
dtypes: float64(18), int64(45), object(47)
memory usage: 135.6+ MB


## DATA CLEANING

In [328]:
### NA Threshold Function
# A function that takes a dataframe and returns the features that are safe to work with
def threshold(data):
    # Renaming dataset for conveniece
    # data = fifa_23 
    
    # The data integrity threshold provided (30%)
    threshold = 0.3
    
    # Calculating threshold count for missing values
    threshold_count = int(threshold * len(data))
    
    # Pulling out unsafe columns
    unsafe_columns = [col for col in data.columns if data[col].isnull().sum() > threshold_count]
    print(f"{len(unsafe_columns)} unsafe columns were identified.")
    
    return unsafe_columns

In [329]:
# Calling the threshold function on my dataframe
threshold(fifa_23)

8 unsafe columns were identified.


['club_loaned_from',
 'nation_team_id',
 'nation_position',
 'nation_jersey_number',
 'release_clause_eur',
 'player_tags',
 'player_traits',
 'goalkeeping_speed']

In [330]:
# Dropping the unsafe features(columns) in the dataframe
dataset = fifa_23.drop(columns = ['club_loaned_from','nation_team_id','nation_position','nation_jersey_number','release_clause_eur','player_tags','player_traits','goalkeeping_speed'])

In [331]:
dataset.columns
# The number of coolumns in the dataframe is now '102'.

Index(['player_id', 'player_url', 'fifa_version', 'fifa_update',
       'fifa_update_date', 'short_name', 'long_name', 'player_positions',
       'overall', 'potential',
       ...
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk',
       'player_face_url'],
      dtype='object', length=102)

In [332]:
# Dropping extra unecessary columns that are irrelevant to player rating
dataset = dataset.drop(columns = ['fifa_update','player_id','player_url','player_positions','dob','fifa_update_date','work_rate','body_type','real_face','short_name','long_name','club_contract_valid_until_year','fifa_version','league_level','nationality_id','player_face_url','league_id', 'league_name','club_team_id', 'club_name', 'club_position', 'club_jersey_number','club_joined_date', 'nationality_name', 'preferred_foot', 'weak_foot','ls', 'st', 'rs','lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm','rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb','rcb', 'rb', 'gk'])
dataset.columns

Index(['overall', 'potential', 'value_eur', 'wage_eur', 'age', 'height_cm',
       'weight_kg', 'skill_moves', 'international_reputation', 'pace',
       'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure', 'defending_marking_awareness',
       'defending_standing_tackle', 'defending_sliding_tackle',
       'goalkeeping_diving', 'goalkeeping_handling', '

In [333]:
# Identifying Columns With Missing Values
# A function that searches for columns in the dataframe with missing values
def find_null_columns(x):
    null_columns = []
    for col in x.columns:
        if x[col].isnull().any():
            null_columns.append(col)
    print(null_columns)

In [334]:
find_null_columns(dataset)
# The displayed list is an array of the columns with missing entriess

['value_eur', 'wage_eur', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'mentality_composure']


In [335]:
# Handling missing values in the null columns 
# Initializing an array to store the columns with null values
columns_to_rectify = ['value_eur', 'wage_eur', 'league_id', 'league_name', 'league_level', 'club_team_id', 'club_name', 'club_position', 'club_jersey_number', 'club_joined_date', 'club_contract_valid_until_year', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'mentality_composure']

# A function that replaces missing values in the columns of a datset with the deian of the column
for column in columns_to_rectify:
    if column in dataset.columns:
        # Fill NAs in each column with the median of that particular column
        dataset[column] = dataset[column].fillna(dataset[column].median())

## CORRELATION MATRIX

In [336]:
correlation_matrix = dataset.select_dtypes(include=[np.number]).corr()

In [337]:
# Mapping correlation on the overall column 
# This enables the inspection of how the overall rating of players varies with other columns
corr_overall = correlation_matrix['overall'].sort_values(ascending=False)

In [338]:
corr_overall

overall                        1.000000
movement_reactions             0.845753
potential                      0.695362
passing                        0.629868
wage_eur                       0.607297
mentality_composure            0.600033
value_eur                      0.562677
dribbling                      0.561034
attacking_short_passing        0.499757
mentality_vision               0.493282
international_reputation       0.485234
skill_long_passing             0.483808
power_shot_power               0.480297
physic                         0.473469
age                            0.458390
skill_ball_control             0.457011
shooting                       0.444746
skill_curve                    0.415863
power_long_shots               0.409086
mentality_aggression           0.398310
attacking_crossing             0.396833
skill_fk_accuracy              0.387269
attacking_volleys              0.377048
skill_dribbling                0.369827
power_stamina                  0.360025


In [339]:
dataset.shape

(161583, 49)

## FEATURE ENGINEERING

#### Feature engineering is the process of creating new features or transforming existing features in a dataset to improve the performance of machine learning models. This involves using domain knowledge, creativity, and various techniques to extract useful information from raw data that can enhance the predictive power of the model.

In [340]:
# Aggregating statistics for mental
mentality_stats = ['mentality_composure','mentality_aggression','mentality_positioning','mentality_vision','mentality_penalties','mentality_interceptions']
dataset['mental'] = dataset[mentality_stats].mean(axis = 1)
dataset.drop(columns = mentality_stats, inplace = True)

In [341]:
# Aggregating statistics for power
power_stats = ['power_shot_power','power_long_shots','power_stamina','power_long_shots','power_strength','power_jumping']
dataset['power'] = dataset[power_stats].mean(axis = 1)
dataset.drop(columns = power_stats, inplace = True)

In [342]:
# Aggregating statistics for skill
skill_stats = ['skill_fk_accuracy','skill_long_passing','skill_ball_control','skill_curve','skill_dribbling','skill_moves']
dataset['skill'] = dataset[skill_stats].mean(axis=1)
dataset.drop(columns = skill_stats, inplace = True)

In [343]:
# Aggregating statistics for goal keeping
goal_keeping_stats = ['goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking','goalkeeping_positioning', 'goalkeeping_reflexes']
dataset['goalkeeping'] = dataset[goal_keeping_stats].mean(axis=1)
dataset.drop(columns = goal_keeping_stats, inplace=True)

In [344]:
# Aggregating statistics for movement
movement_stats = ['movement_acceleration','movement_sprint_speed','movement_agility','movement_reactions','movement_balance']
dataset['movement'] = dataset[movement_stats].mean(axis = 1)
dataset.drop(columns = movement_stats, inplace = True)

In [345]:
# Aggregating statistics for attack
attacking_stats = ['attacking_crossing','attacking_finishing','attacking_heading_accuracy','attacking_short_passing','attacking_volleys']
dataset['attack'] = dataset[attacking_stats].mean(axis = 1)
dataset.drop(columns = attacking_stats, inplace = True)

In [346]:
# Aggregating statistics for defence
defending_stats = ['defending_marking_awareness','defending_standing_tackle','defending_sliding_tackle']
dataset['defence'] = dataset[defending_stats].mean(axis = 1)
dataset.drop(columns = defending_stats, inplace = True)

In [347]:
dataset.info()
dataset.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 21 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   overall                   161583 non-null  int64  
 1   potential                 161583 non-null  int64  
 2   value_eur                 161583 non-null  float64
 3   wage_eur                  161583 non-null  float64
 4   age                       161583 non-null  int64  
 5   height_cm                 161583 non-null  int64  
 6   weight_kg                 161583 non-null  int64  
 7   international_reputation  161583 non-null  int64  
 8   pace                      161583 non-null  float64
 9   shooting                  161583 non-null  float64
 10  passing                   161583 non-null  float64
 11  dribbling                 161583 non-null  float64
 12  defending                 161583 non-null  float64
 13  physic                    161583 non-null  f

(161583, 21)